In [1]:
# things that i am writing are here and popping up hopefully
# We will use this to suppress some warnings that are not important
import warnings

# Suppress specific Pydantic warnings that clutter the output
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# We will use dotenv to read the .env file
from dotenv import load_dotenv
load_dotenv()

# This import will return an error if LiteLLM is not installed 
import litellm
import os

# Use this to measure response time
import time

# URL of Ohio State's LiteLLM proxy server
custom_api_base = "https://litellmproxy.osu-ai.org" 

# Our API key for Astronomy 1221 (keep this private to our class)
astro1221_key = os.getenv("ASTRO1221_API_KEY")
if astro1221_key:
    print("Successfully loaded Astronomy 1221 key")
else:
    print("Error: did not find key. Check that .env exists in the same folder/directory as your class notebooks")

# Check that .gitignore exists in this directory
if os.path.isfile('.gitignore'):
    print("Successfully found .gitignore in the current directory")
else:
    print("Error: Did not find .gitignore. Please download .gitignore from carmen and put in the same folder/directory as your class notebooks.")

with open('.gitignore', 'r') as f:
    content = f.read()
    if '.env' in content:
        print("Confirmed that .gitignore has the .env exclusion")
    else: 
        print("Error: Did not find .env in .gitignore. Please download .gitignore from carmen and put with your class notebooks.")

Error: did not find key. Check that .env exists in the same folder/directory as your class notebooks
Successfully found .gitignore in the current directory
Confirmed that .gitignore has the .env exclusion


In [5]:
def prompt_llm(messages, model="openai/GPT-4.1-mini", temperature=0.2, max_tokens=1000):
    """
    Send a prompt or conversation to an LLM using LiteLLM and return the response.

    Parameters:
        messages: Either a string (single user prompt) or a list of message dicts with
                  "role" and "content". If a string, formatted as [{"role": "user", "content": messages}].
        model (str, optional): The name of the model to use. Defaults to "openai/GPT-4.1-mini".
        temperature (float, optional): Value between 0 and 2; higher values make output more random. Defaults to 0.2.
        max_tokens (int, optional): Maximum number of tokens to generate in the completion. Must be a positive integer. Defaults to 1000.

    Prints the answer returned by the model.
    
    Returns:
        response: The full response object from LiteLLM.

    Raises:
        ValueError: If `temperature` is not in [0, 2] or `max_tokens` is not a positive integer.
    """
    # If messages is a string, format it as a single user message
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
    # Validate temperature
    if not (isinstance(temperature, (int, float)) and 0 <= temperature <= 2):
        raise ValueError("temperature must be a float between 0 and 2 (inclusive).")
    # Validate max_tokens
    if not (isinstance(max_tokens, int) and max_tokens > 0):
        raise ValueError("max_tokens must be a positive integer.")

    try: 
        print("Contacting LLM via University Server...")

        response = litellm.completion(
            model=model,
            messages=messages,
            api_base=custom_api_base,
            api_key=astro1221_key,
            temperature=temperature,
            max_tokens=max_tokens
        )

        answer = response['choices'][0]['message']['content']
        print(f"\nSUCCESS! Here is the answer from {model}:\n")
        print(answer)
        print("\n")

    except Exception as e:
        print(f"\nERROR: Could not connect. Details:\n{e}")    
        response = None

    return response

In [6]:
def show_response_metadata(response):
    '''
    Convert the response to a dictionary
    Print information about token usage and costs
    '''
    
    # Here are the top level keys
    response_dict = response.model_dump()
    # print(f"Top-level keys: {response_dict.keys()}\n")
    
    # Here are more details: 
    # 1. Get the exact model version used by the server
    used_model = response.model
    
    # 2. Extract token counts from the 'usage' attribute
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens
    
    # 3. Calculate the cost (LiteLLM does the math based on current rates)
    cost = litellm.completion_cost(completion_response=response)
    
    print(f"--- Query Metadata ---")
    print(f"Model:        {used_model}")
    print(f"Input Tokens: {input_tokens}")
    print(f"Output Tokens:{output_tokens}")
    print(f"Total Tokens: {total_tokens}")
    print(f"Estimated Cost: ${cost:.6f}") # Showing 6 decimal places for small queries

In [7]:
import csv
import os
import numpy as np

# check path location to make sure directory is correct
current_dir = os.getcwd()

# make sure the file is in our directory
if os.path.exists("astro_jeopardy_answers.csv"):
    print("astro_jeopardy_answers.csv exists in the current directory.")
else:
    print("astro_jeopardy_answers.csv does not exist in the current directory.")

# read the file
with open("astro_jeopardy_answers.csv", "r") as file:
    reader = csv.reader(file)
    try:
        answer = np.genfromtxt("astro_jeopardy_answers.csv", delimiter=",", dtype=str)
        print("Data array created successfully.")
        print(f"The array's shape is {answer.shape}.")
        print(answer)
    except Exception as e:
        print(f"Error creating data array: {e}")


astro_jeopardy_answers.csv exists in the current directory.
Data array created successfully.
The array's shape is (6, 6).
[['Exoplanets' 'Galaxies' 'Constellations' 'Planets' 'Moons'
  'Astronomers/Physicists/Astronauts']
 ['HD189733b' 'the Milky Way' 'Ursa Major' 'Neptune' 'Titan'
  'Robert Henry Lawerence Jr.']
 ['Wasp 12-b' 'M31' 'Orion' 'Saturn' 'Europa' 'Kathrine Johnson']
 ['Tres 2b' 'M32' 'Cassiopeia' 'Venus' 'Enceladus' 'Sally Ride']
 ['GJ 1132b' 'M33' 'Pegasus' 'Uranus' 'Triton' 'Sunita Williams']
 ['51 Pegasi b' 'M64' 'Sagittarius' 'Earth' 'Phobos' 'Benjamin Bennecker']]


In [37]:
import csv
import os
import numpy as np

# check path location to make sure directory is correct
current_dir = os.getcwd()

# make sure the file is in our directory
if os.path.exists("astro_jeopardy_facts.csv"):
    print("astro_jeopardy_facts.csv exists in the current directory.")
else:
    print("astro_jeopardy_facts.csv does not exist in the current directory.")

# read the file
# Note: np.genfromtxt expects the SAME number of columns on every row.
# The header has no @ (1 column), but fact lines have @ at start (2 columns) -> "got 2 columns instead of 1"
# Fix: read line-by-line and extract the fact text (everything after @)
with open("astro_jeopardy_facts.csv", "r") as file:
    facts_list = []
    next(file)  # skip header row
    for line in file:
        line = line.strip()
        if line.startswith("@"):
            facts_list.append(line[1:].strip())  # remove @ and get fact text
        # skip merge conflict lines and other non-fact lines
    facts = np.array(facts_list)
    desired_shape = (5, 6)
    facts2 = np.array(facts_list).reshape(desired_shape)
    print("Data array created successfully.")
    print(f"The array's shape is {facts2.shape}.")
    print(facts2)  # Show first 3 facts

astro_jeopardy_facts.csv exists in the current directory.
Data array created successfully.
The array's shape is (5, 6).
[['HD189733b is a Hot Jupiter exoplanet that is known for its deadly conditions such as: raining molten glass sideways and containing a disgusting smell of rotten egg. This planet also contains an abundant amount of hydrogen sulfide and was discovered through radial velocity.'
  'The Milky Way is the galaxy we live in! The Milky Way spans across more than 100,000 light-years. It takes our solar system around 250 million years to orbit our galaxy once!'
  'Ursa Major is also known as the Great Bear and the third largest constellation. In Greek mythology, this bear is Callisto, a nymph of Artemis that Zeus is madly in love with.'
  'Neptune was first observed in 1612 by Galileo and was later the first planet in our solar system to be located by math in 1846. A typical day on Neptune is around 16 hours, while a typical year is around 60,190 Earth days. Due to the long ye

In [26]:
chat_assignment = f"""You are Alex Trebek hosting a game of Astronomy-themed Jeopardy. Generate one Jeopardy-style
     clue using the given facts, with the answer being {answer[1,2]} in the following category: {answer[0,2]}. 
     Do not mention the answer in the prompt, and only include the clue in your response."""
prompt = "Ursa Major is also known as the Great Bear and the third largest constellation. In Greek mythology, this bear is Callisto, a nymph of Artemis that Zeus is madly in love with."
# Next steps: make a for loop to run through each individual clue and answer in facts array. Then append each one to a 
# list and then make a numpy array. Read astro_jeopardy_facts.csv and also make it into a numpy array using the same 
# method as above. Replace the prompt variable with the corresponding row and column from the numpy array (the one with facts)

messages = [{"role": "system", "content": chat_assignment}, 
{"role": "user", "content": prompt}]

response = prompt_llm(messages)
print(show_response_metadata(response))

Contacting LLM via University Server...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


ERROR: Could not connect. Details:
litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


AttributeError: 'NoneType' object has no attribute 'model_dump'